In [1]:
from antelope_catalog import LcCatalog
cat = LcCatalog('/data/GitHub/2018/variability_jlca/catalog/')

Loading JSON data from /data/GitHub/lca-tools/lcatools/qdb/data/flowable_synlist.json:
Loading JSON data from /data/GitHub/lca-tools/lcatools/qdb/data/quantity_synlist.json:
local.qdb: /data/GitHub/2018/variability_jlca/catalog/reference-quantities.json
Loading JSON data from /data/GitHub/2018/variability_jlca/catalog/reference-quantities.json:
local.qdb: /data/GitHub/lca-tools/lcatools/qdb/data/elcd_reference_quantities.json
25 new quantity entities added (25 total)
6 new flow entities added (6 total)


In [2]:
m_gly = next(cat.query('local.ecoinvent.3.4.apos.index').processes(Name='market for glycerine'))
lci_native = [x for x in m_gly.lci()]

Loading JSON data from /data/GitHub/2018/variability_jlca/catalog/index/a57075ecd68e35920696904ec830bc522fc6b2b2.json.gz:
local.ecoinvent.3.4.apos.index.20190118: /data/GitHub/2018/variability_jlca/catalog/index/a57075ecd68e35920696904ec830bc522fc6b2b2.json.gz
local.ecoinvent.3.4.apos: /data/LCI/Ecoinvent/3.4/ecoinvent 3.4_apos_ecoSpold02
19 new quantity entities added (19 total)
13290 new process entities added (13290 total)
4851 new flow entities added (4851 total)
local.ecoinvent.3.4.apos.index.20190118: /data/GitHub/2018/variability_jlca/catalog/archives/local.ecoinvent.3.4.apos.index.20190118_background.mat
Loading JSON data from /data/GitHub/2018/variability_jlca/catalog/archives/local.ecoinvent.3.4.apos.index.20190118_background.mat.index.json.gz:
completed 65 iterations


In [3]:
%timeit -n4 -r2 [x for x in m_gly.lci()]

completed 65 iterations
completed 65 iterations
completed 65 iterations
completed 65 iterations
completed 65 iterations
completed 65 iterations
completed 65 iterations
completed 65 iterations
281 ms ± 635 µs per loop (mean ± std. dev. of 2 runs, 4 loops each)


In [4]:
[str(x) for x in lci_native[:10]]

['Output:     [1.49e-09 [kg]]     [local.ecoinvent.3.4.apos] Cobalt [kg]',
 'Output:     [3.27e-08 [kg]]     [local.ecoinvent.3.4.apos] Potassium [kg]',
 'Output:     [1.07e-09 [kg]]     [local.ecoinvent.3.4.apos] Silver, ion [kg]',
 'Output:     [2.93e-08 [kg]]     [local.ecoinvent.3.4.apos] Barium [kg]',
 'Output:     [3.27e-08 [kg]]     [local.ecoinvent.3.4.apos] Potassium [kg]',
 'Output:     [2.48e-07 [kg]]     [local.ecoinvent.3.4.apos] Molybdenum [kg]',
 'Output:     [3.78e-08 [kg]]     [local.ecoinvent.3.4.apos] Aluminium [kg]',
 'Output:     [8.72e-07 [kg]]     [local.ecoinvent.3.4.apos] Zinc [kg]',
 'Output:     [6.87e-08 [kg]]     [local.ecoinvent.3.4.apos] Sulfur [kg]',
 'Output:     [4.56e-10 [kg]]     [local.ecoinvent.3.4.apos] PAH, polycyclic aromatic hydrocarbons [kg]']

In [5]:
lci_spsolve = [x for x in m_gly.lci(solver='spsolve')]

/data/virtualenvs/lca-tools/lib/python3.7/site-packages/scikits/umfpack/umfpack.py:721: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 6.75e+12)
  warnings.warn(msg, UmfpackWarning)


In [6]:
%timeit -n4 -r2 [x for x in m_gly.lci(solver='spsolve')]

2.1 s ± 12.6 ms per loop (mean ± std. dev. of 2 runs, 4 loops each)


In [7]:
%time lci_factorize = [x for x in m_gly.lci(solver='factorize')]

CPU times: user 31.5 s, sys: 136 ms, total: 31.6 s
Wall time: 31.7 s


In [8]:
%timeit -n4 -r2 [x for x in m_gly.lci(solver='factorize')]

47 ms ± 3.22 ms per loop (mean ± std. dev. of 2 runs, 4 loops each)


In [9]:
len(lci_factorize)

1992

In [10]:
[(lci_native[i].value, lci_spsolve[i].value, lci_factorize[i].value) for i in range(10, 20)]

[(6.068686215365687e-08, 6.068686973251845e-08, 6.068686973251844e-08),
 (5.9528173027745144e-09, 5.952818046191888e-09, 5.952818046191887e-09),
 (5.500310565529881e-10, 5.500311252318348e-10, 5.500311252318347e-10),
 (1.6801133921052114e-09, 1.6801136019253797e-09, 1.6801136019253793e-09),
 (1.5207889173718232e-10, 1.5207891072898087e-10, 1.5207891072898084e-10),
 (2.3318807452972e-09, 2.331881036511877e-09, 2.3318810365118762e-09),
 (1.0127030696223717e-07, 1.012703555589921e-07, 1.012703555589921e-07),
 (3.791123307071502e-08, 3.79112468335066e-08, 3.791124683350659e-08),
 (8.820547375034896e-07, 8.820547505377273e-07, 8.820547505377272e-07),
 (2.3318807452972006e-10, 2.3318810365118754e-10, 2.331881036511875e-10)]

### Debugging

In [5]:
ar = cat.get_archive(m_gly.origin, 'background')

In [6]:
lci = ar._flat._compute_lci(m_gly.external_ref, m_gly.reference().flow.external_ref)

completed 65 iterations


In [7]:
lci.shape

(2030, 1)

In [8]:
lci2 = ar._flat._compute_lci(m_gly.external_ref, m_gly.reference().flow.external_ref, solver='spsolve')

we are where we are supposed to be


/data/virtualenvs/lca-tools/lib/python3.7/site-packages/scikits/umfpack/umfpack.py:721: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 6.75e+12)
  warnings.warn(msg, UmfpackWarning)


In [12]:
from scipy.sparse.linalg import spsolve

In [13]:
def ima():
    return eye(ar._flat._A.shape[0]) - ar._flat._A

In [23]:
from scipy.sparse import eye, csr_matrix, isspmatrix
from antelope_background.background.flat_background import _unit_column_vector

In [15]:
y = _unit_column_vector(ar._flat.ndim, 4477)

In [16]:
type(y)

scipy.sparse.csr.csr_matrix

In [17]:
y.shape

(11607, 1)

In [18]:
y.ndim

2

In [19]:
y.shape[1]

1

In [20]:
y.toarray()

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=int64)

In [24]:
v = spsolve(ima(), y)

we are where we are supposed to be


/data/virtualenvs/lca-tools/lib/python3.7/site-packages/scikits/umfpack/umfpack.py:721: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 6.75e+12)
  warnings.warn(msg, UmfpackWarning)


In [25]:
y.shape

(11607, 1)

In [26]:
type(v)

numpy.ndarray

In [30]:
v.shape

(11607,)

In [45]:
from scipy.sparse.linalg import factorized

In [47]:
_lu = factorized(ima().tocsc())

In [49]:
v = _lu(y.toarray().flatten())

In [21]:
isspmatrix(y)

NameError: name 'isspmatrix' is not defined

In [52]:
csr_matrix(v).T

<11607x1 sparse matrix of type '<class 'numpy.float64'>'
	with 11607 stored elements in Compressed Sparse Column format>

In [43]:
%timeit -n12 -r4 csr_matrix(v, shape=(1, 11607))

621 µs ± 52.4 µs per loop (mean ± std. dev. of 4 runs, 12 loops each)


In [23]:
help(spsolve)

Help on function spsolve in module scipy.sparse.linalg.dsolve.linsolve:

spsolve(A, b, permc_spec=None, use_umfpack=True)
    Solve the sparse linear system Ax=b, where b may be a vector or a matrix.
    
    Parameters
    ----------
    A : ndarray or sparse matrix
        The square matrix A will be converted into CSC or CSR form
    b : ndarray or sparse matrix
        The matrix or vector representing the right hand side of the equation.
        If a vector, b.shape must be (n,) or (n, 1).
    permc_spec : str, optional
        How to permute the columns of the matrix for sparsity preservation.
        (default: 'COLAMD')
    
        - ``NATURAL``: natural ordering.
        - ``MMD_ATA``: minimum degree ordering on the structure of A^T A.
        - ``MMD_AT_PLUS_A``: minimum degree ordering on the structure of A^T+A.
        - ``COLAMD``: approximate minimum degree column ordering
    use_umfpack : bool, optional
        if True (default) then use umfpack for the solution.  This 

In [21]:
csr_matrix(lci2).shape

(1, 2030)

In [13]:
cat.show_interfaces()

local.ecoinvent.3.4.apos [basic, index, inventory]
local.ecoinvent.3.4.apos.index.20190118 [background, basic, index]
local.ecoinvent.3.4.conseq [basic, index, inventory]
local.ecoinvent.3.4.cutoff [basic, index, inventory]
local.ecoinvent.3.4.cutoff.index.20190118 [background, basic, index]
local.lcia.ecoinvent.3.1 [basic, index, quantity]
local.qdb [basic, index, quantity]


In [2]:
p = next(cat.query('local.ecoinvent.3.4.cutoff').processes())

local.ecoinvent.3.4.cutoff: /data/LCI/Ecoinvent/3.4/ecoinvent 3.4_cutoff_ecoSpold02


In [5]:
for rx in p.references():
    print('%d: %s' % (len([x for x in p.consumers(rx.flow)]), rx.flow))

1: [local.ecoinvent.3.4.cutoff] heat, district or industrial, other than natural gas [MJ]
0: [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]


In [6]:
qc = cat.query('local.ecoinvent.3.4.cutoff')

## Auditing Ecoinvent technosphere design

We want to see two things as "nontrivial consumers":
 - non-market processes with more than one consumer
 - non-market processes with nonmarket consumers

In [74]:
def nontrivial_producer(processes_generator):
    count = list((0, 0))
    for p in processes_generator:
        for rx in p.references():
            cons = [t for t in p.consumers(rx.flow)]
            nmcons = [t for t in cons if not t.termination['Name'].startswith('market')]
            if len(cons) > 1:
                count[0] += 1
                # print(rx)
                # print('%d consumers' % len(cons))
                if len(nmcons) > 0:
                    count[1] += 1
                    # print('%d nonmarket consumers' % len(nmcons))
                # yield len(cons), len(nmcons), rx
            elif len(nmcons) > 0:
                count[1] += 1
                # print('%d nonmarket consumers' % len(nmcons))
            yield len(cons), len(nmcons), rx
    print('%d multiconsumers;; %d nonmarket consumers' % (count[0], count[1]))
                
def nonmarket_producer(processes_generator):
    for x in processes_generator:
        if x['Name'].startswith('market'):
            continue
        yield x

In [75]:
_nmgen = nonmarket_producer(qc.processes())


In [76]:
_npgen = nontrivial_producer(_nmgen)

In [77]:
rxs = [z for z in _npgen]

1804 multiconsumers;; 1838 nonmarket consumers


In [78]:
len(rxs)

10923

In [97]:
sum([k[0] for k in rxs])

15967

In [99]:
sum([k[1] for k in rxs])

5692

In [68]:
rxs.append(next(_npgen))

Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
3 consumers
2 nonmarket consumers


In [69]:
newrxs = [z for z in _npgen]

Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage, aluminium industry [kWh]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
3 consumers
2 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
3 consumers
2 nonmarket consumers
Output: (*) [1 [hour]] [local.ecoinvent.3.4.cutoff] cable yarding [hour]
9 consumers
8 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
3 consumers
2 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] ele

Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
3 consumers
2 nonmarket consumers
Output: (*) [1 [kg]] [local.ecoinvent.3.4.cutoff] copper scrap, sorted, pressed [kg]
78 consumers
77 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
3 consumers
2 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
3 consumers
2 nonmarket consumers
Output: (*) [1 [MJ]] [local.ecoinvent.3.4.cutoff] heat, district or industrial, natural gas [MJ]
6 consumers
5 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cut

Output: (*) [1 [unit]] [local.ecoinvent.3.4.cutoff] mini CHP plant, components for heat only [unit]
3 consumers
2 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kg]] [local.ecoinvent.3.4.cutoff] copper concentrate, sulfide ore [kg]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
3 consumers
2 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
3 consumers
2 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
2 consumers
1 nonmarket consumers
Squashing bad termination in linked reference exchange, 1c56bd91-2ef2-4fa8-a9fb-826e4a8daca4
Flow aa447135-9cc5-4f34-9385-ca2a0ae70d3c Term aa948b8d-c846-4661-8

Output: (*) [1 [kg]] [local.ecoinvent.3.4.cutoff] clinker [kg]
3 consumers
2 nonmarket consumers
Output: (*) [1 [MJ]] [local.ecoinvent.3.4.cutoff] heat, district or industrial, other than natural gas [MJ]
11 consumers
11 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
3 consumers
2 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
2 consumers
1 nonmarket consumers
Squashing bad termination in linked reference exchange, 0675f357-4a24-4f89-ad6f-b48c62ddd9d8
Flow c5651546-a49e-4c1d-b856-1cf358ab466a Term 66962eb8-0962-43f0-bdbe-634d2ba63cfd
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
11 consumers
Output: (*) [1 [kg

Output: (*) [-1 [kg]] [local.ecoinvent.3.4.cutoff] waste plastic, mixture [kg]
9 consumers
8 nonmarket consumers
Output: (*) [1 [MJ]] [local.ecoinvent.3.4.cutoff] heat, district or industrial, other than natural gas [MJ]
3 consumers
2 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
3 consumers
2 nonmarket consumers
Squashing bad termination in linked reference exchange, c24ce0b9-f4b2-4d12-875c-97d08b2be8da
Flow c5651546-a49e-4c1d-b856-1cf358ab466a Term abe2477a-5f31-458e-a3aa-92928702c077
Output: (*) [1 [m3]] [local.ecoinvent.3.4.cutoff] sawnwood, beam, hardwood, raw, dried (u=20%) [m3]
2 consumers
2 nonmarket consumers
Output: (*) [1 [kg]] [local.ecoinvent.3.4.cutoff] potassium fertiliser, as K2O [kg]
2 consumers
2 nonmarket consumers
Output: (*) [1 [kg]] [local.ecoinvent.3.4.cutoff] phosphate fertiliser, as P2O5 [kg]
2 consumers
2 nonmarket consumers
Output: (*) [1 [hour]] [local.ecoinvent.3.4.cutoff] operation, computer, desktop

...

Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
3 consumers
2 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
3 consumers
2 nonmarket consumers
1 nonmarket consumers
Output: (*) [1 [hour]] [local.ecoinvent.3.4.cutoff] wood chipping, chipper, mobile, diesel, at forest road [hour]
10 consumers
9 nonmarket consumers
1 nonmarket consumers
1 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
3 consumers
2 nonmarket consumers
Output: (*) [1 [m3]] [local.ecoinvent.3.4.cutoff] sawnwood, lath, hardwood, raw, dried (u=20%) [m3]
3 consumers
2 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, hi

Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
3 consumers
2 nonmarket consumers
Output: (*) [1 [metric ton*km]] [local.ecoinvent.3.4.cutoff] transport, pipeline, long distance, natural gas [metric ton*km]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kg]] [local.ecoinvent.3.4.cutoff] vinyl acetate [kg]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
7 consumers
6 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
3 consumers
2 nonmarket consumers
Output: (*) [1 [kg]] [local.ecoinvent.3.4.cutoff] aluminium scrap, post-consumer, prepared for melting [kg]
64 consumers
64 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
4 consumers
3 nonmarket consumers
Output: (

Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
3 consumers
2 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
2 consumers
1 nonmarket consumers
Output: (*) [-1 [kg]] [local.ecoinvent.3.4.cutoff] waste wood, untreated [kg]
4 consumers
3 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
2 consumers
2 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high 

Output: (*) [1 [MJ]] [local.ecoinvent.3.4.cutoff] natural gas, burned in gas motor, for storage [MJ]
10 consumers
9 nonmarket consumers
Output: (*) [1 [ha]] [local.ecoinvent.3.4.cutoff] fertilising, by broadcaster [ha]
34 consumers
33 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage, label-certified [kWh]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
3 consumers
2 nonmarket consumers
Output: (*) [1 [ha]] [local.ecoinvent.3.4.cutoff] swath, by rotary windrower [ha]
4 consumers
3 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
3 consumers
2 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]
2 consumers
1 nonmarket consumers
Output: (*) [1 [kWh]] [loca

TypeError: %d format: a number is required, not list

In [16]:
con = _

In [18]:
con[0].termination['Name']

'market for heat, district or industrial, other than natural gas'

In [7]:
str(p.reference())

'Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]'

In [5]:
[str(x) for x in p.references()]

['Output: (*) [1 [kWh]] [local.ecoinvent.3.4.cutoff] electricity, high voltage [kWh]',
 'Output: (*) [1 [MJ]] [local.ecoinvent.3.4.cutoff] heat, district or industrial, other than natural gas [MJ]']

In [4]:
p.show()

ProcessRef catalog reference (6472b6cb-1883-4a14-9b4c-574715250ba3)
origin: local.ecoinvent.3.4.cutoff
UUID: 6472b6cb-1883-4a14-9b4c-574715250ba3
   Name: heat and power co-generation, diesel, 200kW electrical, SCR-NOx reduction
Comment: The multioutput-process 'diesel, burned in cogen 200kWe diesel SCR, allocation exergy' delivers the coproducts 'heat, at cogen 200kWe diesel SCR, allocation exergy' and 'electricity, at cogen 200kWe diesel SCR, allocation exergy'. The exergy allocation is the allocation scheme suggested to be used within ecoinvent (e.g. in electricity mixes).
[This dataset was already contained in the ecoinvent database version 2. It was not individually updated during the transfer to ecoinvent version 3. Life Cycle Impact Assessment results may still have changed, as they are affected by changes in the supply chain, i.e. in other datasets. This dataset was generated following the ecoinvent quality guidelines for version 2. It may have been subject to central changes d

In [8]:
[x.termination['Name'] for x in cat.query(p.origin).consumers(p.external_ref)]

['market for heat, district or industrial, other than natural gas']

In [94]:
rx.value

1.0

In [95]:
next(cat.gen_interfaces(p.origin, 'inventory'))

## LXML / objectify Stuff

In [100]:
import scikits

ModuleNotFoundError: No module named 'scikits'

In [82]:
from antelope_catalog.providers.xml_widgets import *

In [84]:
ar = cat.get_archive('local.ecoinvent.3.4.cutoff', 'inventory')

In [85]:
o = ar.objectify(p.external_ref, p.reference().flow.external_ref)

In [92]:
o.

2

In [15]:
fd = find_tag(o, 'flowData')

<Element {http://www.EcoInvent.org/EcoSpold02}flowData at 0x7ffa106e6c48>

In [16]:
_xg = fd.iterchildren()

In [17]:
exch = next(_xg)

In [38]:
exch = next(_xg)
'; '.join([str(c) for c in exch.iterchildren() if c.tag == '{%s}comment' % o.nsmap[None]])

StopIteration: 

In [39]:
from antelope_reports.tables import AllocationGrid

In [41]:
AllocationGrid(ar, p, report_unallocated=False).dataframe()

,Direction,Ref,C0,C1,Flow,Comment
0,Output,{*},,1,[kWh] [local.ecoinvent.3.4.cutoff] electricity...,EcoSpold01Location=CH; Amount was scaled to th...
1,Output,{*},1,,"[MJ] [local.ecoinvent.3.4.cutoff] heat, distri...",EcoSpold01Location=CH; Amount was scaled to 1.
2,Input,,0.00835,0.183,[kg] market for diesel [CH],EcoSpold01Location=CH; Amount was scaled to th...
3,Input,,2.39e-05,0.000525,[kg] market for lubricating oil [GLO],EcoSpold01Location=RER; vender information\nAm...
4,Input,,0.000285,0.00627,"[kg] market for urea, as N [GLO]",different references; uncertainty of urea need...
5,Input,,-2.39e-05,-0.000525,[kg] market for waste mineral oil [CH],EcoSpold01Location=CH; rough estimate\nAmount ...
6,Input,,1.93e-09,4.25e-08,[unit] market for heat and power co-generation...,EcoSpold01Location=RER; Amount was scaled to t...
7,Input,,1.93e-09,4.25e-08,[unit] market for heat and power co-generation...,Amount was scaled to the amount of reference p...
8,Input,,1.93e-09,4.25e-08,[unit] market for heat and power co-generation...,EcoSpold01Location=RER; Amount was scaled to t...
9,Output,,3.57e-07,7.84e-06,[kg] Ammonia (CAS 007664-41-7) [urban air clos...,estimate based on different references\nAmount...
